In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\spark-3.4.0-bin-hadoop3'

In [2]:
import pyspark
from pyspark.sql import SparkSession

# Set up Spark session
spark = SparkSession.builder \
    .appName("Join Text Files with Condition") \
    .getOrCreate()

In [3]:
# Read input files as RDDs
viewership_rdd = spark.sparkContext.textFile(r"D:\Gurudeep\2 MEC\Subjects\3 Year\2 Big data\Project\Team 12\input2.txt")
shows_rdd = spark.sparkContext.textFile(r"D:\Gurudeep\2 MEC\Subjects\3 Year\2 Big data\Project\Team 12\input1.txt")

In [4]:
# Split lines and convert to key-value pairs
viewership_pairs_rdd = viewership_rdd.map(lambda line: line.split(",")).map(lambda cols: (cols[0], cols[1]))
shows_pairs_rdd = shows_rdd.map(lambda line: line.split(",")).map(lambda cols: (cols[0], cols[1]))

In [5]:
print(viewership_pairs_rdd.take(1))

[('ShowName', 'Viewer')]


In [6]:
print(shows_pairs_rdd.take(1))

[('Show', 'Channel')]


In [7]:
# Perform join operation
joined_rdd = viewership_pairs_rdd.join(shows_pairs_rdd)

In [8]:
joined_rdd.take(1)

[('Cold_Talking', ('287', 'CNO'))]

In [9]:
# Filter by channel
filtered_rdd = joined_rdd.filter(lambda row: row[1][1] == "ABC")

In [12]:
filtered_rdd.take(5)

[('Baked_Games', ('459', 'ABC')),
 ('Baked_Games', ('200', 'ABC')),
 ('Baked_Games', ('1033', 'ABC')),
 ('Baked_Games', ('130', 'ABC')),
 ('Baked_Games', ('207', 'ABC'))]

In [11]:
# Select desired columns
wanted_rdd = filtered_rdd.map(lambda row: (row[1][0], (row[1][1])))

In [15]:
wanted_rdd = filtered_rdd.map(lambda x: (x[0], int(x[1][0])))

In [16]:
wanted_rdd.take(1)

[('Baked_Games', 459)]

In [17]:
# Perform sum aggregation
grouped_rdd = wanted_rdd.reduceByKey(lambda a, b: a + b)

In [19]:
grouped_rdd.take(10)

[('Baked_Games', 9692),
 ('Dumb_Show', 9956),
 ('Hot_Show', 8036),
 ('Hot_Games', 8716),
 ('PostModern_Games', 8244),
 ('Hourly_Cooking', 8452),
 ('Surreal_News', 8024),
 ('Hourly_Talking', 19704),
 ('Almost_Show', 8532),
 ('Cold_Sports', 9636)]

In [22]:
formatted_rdd = grouped_rdd.map(lambda x: f"('{x[0]}', {x[1]})")

In [24]:
# Repartition the RDD to a single partition
single_partition_rdd = formatted_rdd.repartition(1)

# Collect the RDD to the driver as a list
result_list = single_partition_rdd.collect()

# Specify the output file path
output_path = "output.txt"

# Write the list to the output file
with open(output_path, "w") as f:
    for item in result_list:
        f.write(item + "\n")

In [25]:
spark.stop()